# Freespace
- import data [x]
- set up representation yaw, pitch, depth [x]
- solver resolution - nearby points creates boundary? [x]
- visualize on example [x]


In [36]:
import numpy as np
import matplotlib.pyplot as plt
import torch

from vis.deprecated_vis import *
from data.PATHS import TMP_VIS_PATH
from data.NSF_data import NSF_dataset
from data.range_image import range_image_coords, create_depth_img

dataset = NSF_dataset()
data = next(dataset.__iter__())


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

pc1, pc2, gt_flow = data

pc1 = pc1.to(device)
pc2 = pc2.to(device)
gt_flow = gt_flow.to(device)
est_flow = torch.randn(pc1.shape, device=device, requires_grad=True)

In [37]:
from pytorch3d.ops.knn import knn_points

def calculate_polar_coords(pc):

    calc_depth = torch.norm(pc[:,:3], dim=1)
    yaw = - torch.arctan2(pc[:, 1], pc[:, 0])
    pitch = torch.arcsin(pc[:, 2] / (calc_depth + 1e-8))

    return yaw, pitch, calc_depth


def yaw_pitch_depth_to_xyz(yaw, pitch, depth):
    # generated, maybe shifted
    # Convert degrees to radians
    # yaw = math.radians(yaw)
    # pitch = math.radians(pitch)

    # correct [x]
    # Calculate XYZ coordinates
    x = depth * torch.cos(yaw) * torch.cos(pitch)
    y = depth * torch.sin(yaw) * torch.cos(pitch)
    z = depth * torch.sin(pitch)

    xyz = torch.stack([x, y, z]).T#[..., [2, 0, 1]]
    xyz[:, 1] = - xyz[:, 1]

    return xyz



# Representation

# Create freespace label

In [38]:
t_z1 = 1.640420
t_z2 = 1.565252
t_z_mean = (t_z1 + t_z2) / 2
# todo tohle asi zpusobuje error v rekonstrukci, dva body maji diky jinemu souradnemu systemu stejny pitch a yaw, a pak jsou za sebou a jeden ma depth malou a druhy velkou. proto se do point cloudu promitne jinak. - Ale proc range image je jiny, kdyz vstupy jsou stejne?!?

# resolution based on the data?
# todo pouzit puvodni point cloud []





In [197]:

class SmoothnessLoss(torch.nn.Module):

#     # use normals to calculate smoothness loss
    def __init__(self, pc1, pc2=None, K=12, sm_normals_K=0, smooth_weight=1., VA=False, max_radius=2, loss_norm=1, forward_weight=0., pc2_smooth=False, **kwargs):

        super().__init__()
        self.K = K
        self.max_radius = max_radius
        self.pc1 = pc1
        self.pc2 = pc2
        self.normals_K = sm_normals_K
        self.loss_norm = loss_norm
        self.smooth_weight = smooth_weight

        # normal Smoothness
        if self.normals_K > 3:
            self.dist1, self.NN_pc1, _ = self.KNN_with_normals(pc1)
        else:
            self.dist1, self.NN_pc1, _ = knn_points(self.pc1, self.pc1, K=self.K)

        self.NN_pc1 = mask_NN_by_dist(self.dist1, self.NN_pc1, max_radius)


        # vis-aware - jenom skrtam, muze byt po radiusu
        self.VA = VA
        if VA:
            # todo
            pass
        # ff
        self.forward_weight = forward_weight

        # ff with NNpc2
        self.pc2_smooth = pc2_smooth
        self.NN_pc2 = None

        if pc2_smooth:

            if self.normals_K > 3:
                self.dist2, self.NN_pc2, _ = self.KNN_with_normals(pc2)
            else:
                self.dist2, self.NN_pc2, _ = knn_points(self.pc2, self.pc2, K=self.K)

            self.NN_pc2 = mask_NN_by_dist(self.dist2, self.NN_pc2, max_radius)

            if VA:
                pass

    def forward(self, pc1, est_flow, pc2):

        loss = torch.tensor(0, dtype=torch.float32, device=pc1.device)

        if self.smooth_weight > 0:
            smooth_loss, pp_smooth_loss = self.smoothness_loss(est_flow, self.NN_pc1, self.loss_norm)

            loss += self.smooth_weight * smooth_loss

        if self.forward_weight > 0:
            forward_loss, pp_forward_loss = self.forward_smoothness(pc1, est_flow, pc2)

            loss += self.forward_weight * forward_loss

        return loss

    def KNN_with_normals(self, pc):

        normals = estimate_pointcloud_normals(pc, neighborhood_size=self.normals_K)
        pc_with_norms = torch.cat([pc, normals], dim=-1)

        return knn_points(pc_with_norms, pc_with_norms, K=self.K)

    def smoothness_loss(self, est_flow, NN_idx, loss_norm=1, mask=None):

        bs, n, c = est_flow.shape

        if bs > 1:
            print("Smoothness Maybe not working for bs>1, needs testing!")
        K = NN_idx.shape[2]

        est_flow_neigh = est_flow.view(bs * n, c)
        est_flow_neigh = est_flow_neigh[NN_idx.view(bs * n, K)]

        est_flow_neigh = est_flow_neigh[:, 1:K + 1, :]
        flow_diff = est_flow.view(bs * n, c) - est_flow_neigh.permute(1, 0, 2)

        flow_diff = (flow_diff).norm(p=loss_norm, dim=2)
        smooth_flow_loss = flow_diff.mean()
        smooth_flow_per_point = flow_diff.mean(dim=0).view(bs, n)

        return smooth_flow_loss, smooth_flow_per_point


    def forward_smoothness(self, pc1, est_flow, pc2):


        _, forward_nn, _ = knn_points(pc1 + est_flow, pc2, lengths1=None, lengths2=None, K=1, norm=1)

        a = est_flow[0] # magnitude

        ind = forward_nn[0] # more than one?

        if pc1.shape[1] < pc2.shape[1]:
            shape_diff = pc2.shape[1] - ind.shape[0] + 1 # one for dummy    # what if pc1 is bigger than pc2?
            a = torch.nn.functional.pad(a, (0,0,0, shape_diff), mode='constant', value=0)
            a.retain_grad() # padding does not retain grad, need to do it manually. Check it

            ind = torch.nn.functional.pad(ind, (0,0,0, shape_diff), mode='constant', value=pc2.shape[1])  # pad with dummy not in orig

        # storage of same points
        vec = torch.zeros(ind.shape[0], 3, device=pc1.device)

        # this is forward flow withnout NN_pc2 smoothness
        vec = vec.scatter_reduce_(0, ind.repeat(1,3), a, reduce='mean', include_self=False)

        forward_flow_loss = torch.nn.functional.mse_loss(vec[ind[:,0]], a, reduction='none').mean(dim=-1)

        if self.pc2_smooth:
            # rest is pc2 smoothness with pre-computed NN
            keep_ind = ind[ind[:,0] != pc2.shape[1] ,0]

            # znamena, ze est flow body maji tyhle indexy pro body v pc2 a ty indexy maji mit stejne flow.
            n = self.NN_pc2[0, keep_ind, :]

            # beware of zeros!!!
            connected_flow = vec[n] # N x KSmooth x 3 (fx, fy, fz)

            prep_flow = est_flow[0].unsqueeze(1).repeat_interleave(repeats=self.K, dim=1) # correct

            # smooth it, should be fine
            flow_diff = prep_flow - connected_flow  # correct operation, but zeros makes problem

            occupied_mask = connected_flow.all(dim=2).repeat(3,1,1).permute(1,2,0)

            # occupied_mask
            per_flow_dim_diff = torch.masked_select(flow_diff, occupied_mask)

            # per_point_loss = per_flow_dim_diff.norm(dim=-1).mean()
            NN_pc2_loss = (per_flow_dim_diff ** 2).mean()    # powered to 2 because norm will sum it directly

        else:
            NN_pc2_loss = torch.tensor(0.)

        forward_loss = forward_flow_loss.mean() + NN_pc2_loss

        return forward_loss, forward_flow_loss

class VAChamferLoss(torch.nn.Module):

    def __init__(self, pc2, fov_up, fov_down, proj_H, proj_W, max_range, nn_weight=1, nn_max_radius=5, both_ways=False, free_weight=0, margin=0.001, ch_normals_K=0, **kwargs):
        super().__init__()

        self.pc2 = pc2

        # visibility component
        # todo option of "pushing" points out of the freespace
        self.free_weight = free_weight
        self.fov_up = fov_up
        self.fov_down = fov_down
        self.proj_H = proj_H
        self.proj_W = proj_W
        self.max_range = max_range
        self.margin = margin

        # NN component
        self.normals_K = ch_normals_K
        self.nn_weight = nn_weight
        self.nn_max_radius = nn_max_radius
        self.both_ways = both_ways

        torch.use_deterministic_algorithms(mode=True, warn_only=False)  # this ...
        pc2_depth, idx_w, idx_h, inside_range_img = range_image_coords(pc2[0], fov_up, fov_down, proj_H, proj_W)
        self.range_depth = create_depth_img(pc2_depth, idx_w, idx_h, proj_H, proj_W, inside_range_img)
        torch.use_deterministic_algorithms(mode=False, warn_only=False)  # this ...

    def forward(self, pc1, est_flow, pc2=None):
        '''

        Args:
            pc1:
            est_flow:

        Returns:
        mask whether the deformed point cloud is in freespace visibility area
        '''
        # dynamic

        # assign Kabsch to lonely points or just push them out of freespace?
        # precompute chamfer, radius
        chamf_x, chamf_y = self.chamfer_distance_loss(pc1 + est_flow, self.pc2, both_ways=self.both_ways, normals_K=self.normals_K)

        if self.free_weight > 0:
            freespace_loss = self.flow_freespace_loss(pc1, est_flow, chamf_x)

        else:
            freespace_loss = torch.zeros_like(chamf_x, dtype=torch.float32, device=chamf_x.device)

        chamf_loss = self.nn_weight * (chamf_x.mean() + chamf_y.mean()) + self.free_weight * freespace_loss.mean()

        return chamf_loss, freespace_loss


    def chamfer_distance_loss(self, x, y, x_lengths=None, y_lengths=None, both_ways=False, normals_K=0, loss_norm=1):
        '''
        Unique Nearest Neighboors?
        :param x:
        :param y:
        :param x_lengths:
        :param y_lengths:
        :param reduction:
        :return:
        '''
        if normals_K >= 3:
            normals1 = estimate_pointcloud_normals(x, neighborhood_size=normals_K)
            normals2 = estimate_pointcloud_normals(y, neighborhood_size=normals_K)

            x = torch.cat([x, normals1], dim=-1)
            y = torch.cat([y, normals2], dim=-1)


        x_nn = knn_points(x, y, lengths1=x_lengths, lengths2=y_lengths, K=1, norm=loss_norm)
        cham_x = x_nn.dists[..., 0]  # (N, P1)
        # x_nearest_to_y = x_nn[1]

        if both_ways:
            y_nn = knn_points(y, x, lengths1=y_lengths, lengths2=x_lengths, K=1, norm=loss_norm)
            cham_y = y_nn.dists[..., 0]  # (N, P2)
            # y_nearest_to_x = y_nn[1]
        else:

            cham_y = torch.tensor(0, dtype=torch.float32, device=x.device)

        return cham_x, cham_y

    def flow_freespace_loss(self, pc1, est_flow, chamf_x):

        flow_depth, flow_w, flow_h, flow_inside = range_image_coords((pc1 + est_flow)[0], self.fov_up, self.fov_down, self.proj_H, self.proj_W)

            # use it only for flow inside the image
        masked_pc2_depth = self.range_depth[flow_h[flow_inside], flow_w[flow_inside]]
        compared_depth = masked_pc2_depth - flow_depth[flow_inside]

        # compared_depth
        # if flow point before the visible point from pc2, then it is in freespace
        # margin is just little number to not push points already close to visible point
        flow_in_freespace = compared_depth > 0 + self.margin


        # Indexing flow in freespace
        freespace_mask = torch.zeros_like(chamf_x, dtype=torch.bool)[0]
        freespace_mask[flow_inside] = flow_in_freespace

        # only pc1 NN
        freespace_loss = freespace_mask * chamf_x

        return freespace_loss


class LossModule(torch.nn.Module):

    def __init__(self, **kwargs):
        super().__init__()
        self.kwargs = kwargs


    def update(self, pc1, pc2):

        self.VAChamfer_loss = VAChamferLoss(pc2, **self.kwargs)
        self.Smoothness_loss = SmoothnessLoss(pc1, pc2, **self.kwargs)

    def forward(self, pc1, est_flow, pc2):

        chamf_loss, pp_freespace_loss = self.VAChamfer_loss(pc1, est_flow)
        smooth_loss = self.Smoothness_loss(pc1, est_flow, pc2)

        loss = smooth_loss + chamf_loss

        return loss


kwargs = {
    'fov_up': fov_up,
    'fov_down': fov_down,
    'proj_H': proj_H,
    'proj_W': proj_W,
    'max_range': max_range,
    'margin': 0.01,
    'both_ways' : True,

    # Smooth
    'sm_normals_K' : 8,
    'K': 5,
    'forward_weight' : 1,
    'smooth_weight' : 1,

    # Chamfer
    'free_weight' : 1,
    'ch_normals_K' : 0,
    'chamfer_weight' : 2,
    'nn_max_radius' : 2,
    'smoothness_weight' : 0.1,
}

loss_module = LossModule(**kwargs)
loss_module.update(pc1, pc2)
loss = loss_module(pc1, est_flow, pc2)

loss.backward()

# visualize_points3D(pc1[0], pp_freespace_loss[0])



In [202]:
import yaml
with open('../configs/first.yml') as f:
    config = yaml.load(f, Loader=yaml.FullLoader)

print(config['Loss'])


{'fov_up': 'fov_up,', 'fov_down': 'fov_down,', 'proj_H': 'proj_H,', 'proj_W': 'proj_W,', 'max_range': 'max_range,', 'margin': '0.01,', 'both_ways': 'True,', 'sm_normals_K': '8,', 'K': '5,', 'forward_weight': '1,', 'smooth_weight': '1,', 'free_weight': '1,', 'ch_normals_K': '0,', 'nn_weights': '2,', 'nn_max_radius': '2,', 'smoothness_weight': '0.1,'}
